In [1]:
import torch
import numpy as np
import os

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
# torch.set_default_device(device)

DATASET = "./DATASET"

# check if dataset is available
if not os.path.exists(DATASET):
    print("Dataset not found. Please download it from https://www.kaggle.com/datasets/techsash/waste-classification-data and extract it to ./DATASET")
    exit(1)


Device: cuda


In [2]:
# format:
# Organic: DATASET/TRAIN/O/...
# Recyclable: DATASET/TRAIN/R/...

# load dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(DATASET + "/TRAIN", transform=transform)
test_dataset = datasets.ImageFolder(DATASET + "/TEST", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# check dataset
print(f"Train dataset: {len(train_dataset)}")
print(f"Test dataset: {len(test_dataset)}")

# check classes
print(f"Classes: {train_dataset.classes}")

# Function to extract features and labels from a dataset
def extract_features_labels(loader):
    features = []
    labels = []
    for images, targets in loader:
        features.append(images.view(images.size(0), -1).numpy())
        labels.append(targets.numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

# Extract features and labels
train_features, train_labels = extract_features_labels(train_loader)
test_features, test_labels = extract_features_labels(test_loader)



Train dataset: 22564
Test dataset: 2513
Classes: ['O', 'R']


In [3]:
# Method 1: Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier()
dt_clf.fit(train_features, train_labels)
dt_predictions = dt_clf.predict(test_features)
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f'Decision Tree Accuracy: {dt_accuracy:.4f}')


Decision Tree Accuracy: 0.7549


In [4]:
# Method 2: Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(train_features, train_labels)
rf_predictions = rf_clf.predict(test_features)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy: {rf_accuracy:.4f}')



Random Forest Accuracy: 0.8663


In [10]:
from torch import nn, optim
import torch.nn.functional as F
import tqdm

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 9, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(9, 27, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(27, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 4 * 4, 128) # 64*4*4 comes from the shape of the output of conv3: 64 channels and 4x4 image size
        self.fc2 = nn.Linear(128, 2)  # 2 classes: Organic and Recyclable

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

# Initialize the network, loss function, and optimizer
cnn = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

# Training the CNN
num_epochs = 20
for epoch in tqdm.tqdm(range(num_epochs)):
    cnn.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluating the CNN
cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
cnn_accuracy = correct / total
print(f'CNN Accuracy: {cnn_accuracy:.4f}')

  5%|▌         | 1/20 [00:16<05:21, 16.95s/it]

Epoch [1/20], Loss: 0.5186


 10%|█         | 2/20 [00:33<05:02, 16.81s/it]

Epoch [2/20], Loss: 0.5003


 15%|█▌        | 3/20 [00:51<04:54, 17.30s/it]

...

 95%|█████████▌| 19/20 [05:37<00:17, 17.58s/it]

Epoch [19/20], Loss: 0.4227


100%|██████████| 20/20 [05:55<00:00, 17.76s/it]

Epoch [20/20], Loss: 0.4197


CNN Accuracy: 0.8866


In [6]:
from torchvision import models

# Load a pretrained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Remove the classifier part of the model to get the features
vgg16 = nn.Sequential(*list(vgg16.children())[:-1])

# Set the model to evaluation mode
vgg16.eval()

# Function to extract features using VGG16
def extract_features(loader):
    features = []
    labels = []
    with torch.no_grad():
        for images, targets in loader:
            output = vgg16(images)
            features.append(output.view(images.size(0), -1).numpy())
            labels.append(targets.numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

# Extract features and labels
train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

c:\Users\Ben\Documents\code\py\datamining_examples\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ben\Documents\code\py\datamining_examples\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# print features shape
print(f"Train features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")

Train features shape: (22564, 25088)
Test features shape: (2513, 25088)


In [7]:
# Decision Tree Classifier with VGG16 features
dt_clf = DecisionTreeClassifier()
dt_clf.fit(train_features, train_labels)
dt_predictions = dt_clf.predict(test_features)
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f'Decision Tree Accuracy (with feature extraction): {dt_accuracy:.4f}')

Decision Tree Accuracy (with feature extraction): 0.7887


In [11]:
# Random Forest Classifier with VGG16 features
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(train_features, train_labels)
rf_predictions = rf_clf.predict(test_features)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy (with feature extraction): {rf_accuracy:.4f}')

Random Forest Accuracy (with feature extraction): 0.8695


In [12]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensionality
pca = PCA(n_components=32)  # new dimensionality
train_features_pca = pca.fit_transform(train_features)
test_features_pca = pca.transform(test_features)

print(f"Train features shape after PCA: {train_features_pca.shape}")

Train features shape after PCA: (22564, 32)


In [14]:
# Decision Tree Classifier with PCA features
dt_clf = DecisionTreeClassifier()
dt_clf.fit(train_features_pca, train_labels)
dt_predictions = dt_clf.predict(test_features_pca)
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f'Decision Tree Accuracy (with PCA features): {dt_accuracy:.4f}')

Decision Tree Accuracy (with PCA features): 0.7581


In [15]:
# Random Forest Classifier with PCA features
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(train_features_pca, train_labels)
rf_predictions = rf_clf.predict(test_features_pca)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy (with PCA features): {rf_accuracy:.4f}')

Random Forest Accuracy (with PCA features): 0.8548


In [22]:
from sklearn.decomposition import PCA

# Apply PCA with 128 components
pca_128 = PCA(n_components=128)
train_features_pca_128 = pca_128.fit_transform(train_features)
test_features_pca_128 = pca_128.transform(test_features)

print(f"Train features shape after PCA_128: {train_features_pca_128.shape}")

Train features shape after PCA_128: (22564, 128)


In [23]:
# Decision Tree Classifier with PCA_128 features
dt_clf = DecisionTreeClassifier()
dt_clf.fit(train_features_pca_128, train_labels)
dt_predictions = dt_clf.predict(test_features_pca_128)
dt_accuracy = accuracy_score(test_labels, dt_predictions)
print(f'Decision Tree Accuracy (with PCA_128 features): {dt_accuracy:.4f}')

Decision Tree Accuracy (with PCA_128 features): 0.7433


In [24]:
# Random Forest Classifier with PCA_128 features
rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(train_features_pca_128, train_labels)
rf_predictions = rf_clf.predict(test_features_pca_128)
rf_accuracy = accuracy_score(test_labels, rf_predictions)
print(f'Random Forest Accuracy (with PCA_128 features): {rf_accuracy:.4f}')

Random Forest Accuracy (with PCA_128 features): 0.8488
